In [1]:
from pyspark import SparkConf, SparkContext
from pyspark.streaming import StreamingContext


conf = SparkConf().setAppName("Steaming file").setMaster('local')
sc = SparkContext(conf=conf)
ssc = StreamingContext(sc, 2)

NameError: name 'sc' is not defined

# 基本输入源

- 文件流
- 套接字流
- RDD队列流


In [6]:
# 文件流
ssc.textFileStream?

## 套接字流

In [1]:
# 套接字流
ssc.socketTextStream?

Object `ssc.socketTextStream` not found.


服务端
```
nc lk 9999 
```
输入信息

客户端 
```
spark-submit streaming_socket.py localhost 9999

```
统计词频

自定义socket数据

```
import socket

server = socket.socket()

server.bind(('localhost', 9999))
server.listen(1)

while True:
    print("Wating for connect ...")
    # 这里用两个值接受，因为连接上之后使用的是客户端发来请求的这个实例
    # 所以下面的传输要使用conn实例操作
    conn, addr = server.accept()  # 阻塞, 等待连接

    print(f"Connect success! Connection is from {addr[0]}")

    print("Sending data ...")
    conn.send('I love hadoop I love spark hadoop is good spark is fast'.encode())
    # 结束 断开连接, while循环下一次连接
    conn.close()
    print('Connection is broken.')
```

## RDD 队列流

在调试Spark Streaming应用程序的时候，我们可以使用streamingContext.queueStream(queueOfRDD)创建基于RDD队列的DStream


In [ ]:
sc.parallelize()

# 高级数据源
## Kafka

spark 2.3 之后不支持了

# 转换操作

## DStream无状态转换操作

- map(func) ：对源DStream的每个元素，采用func函数进行转换，得到一个新的Dstream

- flatMap(func)： 与map相似，但是每个输入项可用被映射为0个或者多个输出项
- filter(func)： 返回一个新的DStream，仅包含源DStream中满足函数func的项
- repartition(numPartitions)： 通过创建更多或者更少的分区改变DStream的并行程度
- reduce(func)：利用函数func聚集源DStream中每个RDD的元素，返回一个包含单元素RDDs的新DStream
- count()：统计源DStream中每个RDD的元素数量
- union(otherStream)： 返回一个新的DStream，包含源DStream和其他DStream的元素

- countByValue()：应用于元素类型为K的DStream上，返回一个（K，V）键值对类型的新DStream，每个键的值是在原DStream的每个RDD中的出现次数

- reduceByKey(func, [numTasks])：当在一个由(K,V)键值对组成的DStream上执行该操作时，返回一个新的由(K,V)键值对组成的DStream，每一个key的值均由给定的recuce函数（func）聚集起来

- join(otherStream, [numTasks])：当应用于两个DStream（一个包含（K,V）键值对,一个包含(K,W)键值对），返回一个包含(K, (V, W))键值对的新Dstream

- cogroup(otherStream, [numTasks])：当应用于两个DStream（一个包含（K,V）键值对,一个包含(K,W)键值对），返回一个包含(K, Seq[V], Seq[W])的元组

- transform(func)：通过对源DStream的每个RDD应用RDD-to-RDD函数，创建一个新的DStream。支持在新的DStream中做任何RDD操作



无状态转换操作实例：
之前“套接字流”部分介绍的词频统计，就是采用无状态转换，每次统计，都是只统计当前批次到达的单词的词频，和之前批次无关，不会进行累计


## DStream有状态转换操作

1. 滑动窗口转换操作
- 事先设定一个滑动窗口的长度（也就是窗口的持续时间）
- 设定滑动窗口的时间间隔（每隔多长时间执行一次计算），让窗口按照指定时间间隔在源DStream上滑动
- 每次窗口停放的位置上，都会有一部分Dstream（或者一部分RDD）被框入窗口内，形成一个小段的Dstream
- 可以启动对这个小段DStream的计算

一些窗口转换操作的含义：

- window(windowLength, slideInterval) 基于源DStream产生的窗口化的批数据，计算得到一个新的Dstream

- countByWindow(windowLength, slideInterval) 返回流中元素的一个滑动窗口数

- reduceByWindow(func, windowLength, slideInterval) 返回一个单元素流。利用函数func聚集滑动时间间隔的流的元素创建这个单元素流。函数func必须满足结合律，从而可以支持并行计算

- countByValueAndWindow(windowLength, slideInterval, [numTasks]) 当应用到一个(K,V)键值对组成的DStream上，返回一个由(K,V)键值对组成的新的DStream。每个key的值都是它们在滑动窗口中出现的频率


- reduceByKeyAndWindow(func, windowLength, slideInterval, [numTasks]) 应用到一个(K,V)键值对组成的DStream上时，会返回一个由(K,V)键值对组成的新的DStream。每一个key的值均由给定的reduce函数(func函数)进行聚合计算。注意：在默认情况下，这个算子利用了Spark默认的并发任务数去分组。可以通过numTasks参数的设置来指定不同的任务数

- reduceByKeyAndWindow(func, invFunc, windowLength, slideInterval, [numTasks]) 更加高效的reduceByKeyAndWindow，每个窗口的reduce值，是基于先前窗口的reduce值进行增量计算得到的；它会对进入滑动窗口的新数据进行reduce操作，并对离开窗口的老数据进行“逆向reduce”操作。但是，只能用于“可逆reduce函数”，即那些reduce函数都有一个对应的“逆向reduce函数”（以InvFunc参数传入）



## updateStateByKey操作

需要在跨批次之间维护状态时，就必须使用updateStateByKey操作

```
updateStateByKey(updateFunc, numPartitions=None, initialRDD=None)
Return a new “state” DStream where the state for each key is updated by applying the given function on the previous state of the key and the new values of the key.

Parameters
updateFunc – State update function. If this function returns None, then corresponding state key-value pair will be eliminated.
```

# 输出操作

把DStream输出到文本文件中

`running_counts.saveAsTextFiles`

把DStream写入到MySQL数据库中

1. MYSQL 中创建 wordcount表
```
MariaDB [(none)]> create database spark;
Query OK, 1 row affected (0.003 sec)

MariaDB [(none)]> use spark
Database changed
MariaDB [spark]> create table wordcount(word varchar(20), count int(4));

```


# Structed Streaming

Structured Streaming的关键思想是将实时数据流视为一张正在不断添加数据的表

可以把流计算等同于在一个静态表上的批处理查询，Spark会在不断添加数据的无界输入表上运行计算，并进行增量查询

## 微批处理
Structured Streaming默认使用微批处理执行模型，这意味着Spark流计算引擎会定期检查流数据源，并对自上一批次结束后到达的新数据执行批量查询

数据到达和得到处理并输出结果之间的延时超过100毫秒

## 持续处理

Spark从2.3.0版本开始引入了持续处理的试验性功能，可以实现流计算的毫秒级延迟

在持续处理模式下，Spark不再根据触发器来周期性启动任务，而是启动一系列的连续读取、处理和写入结果的长时间运行的任务


## Structured Streaming和Spark SQL、Spark Streaming关系

- Structured Streaming处理的数据跟Spark Streaming一样，也是源源不断的数据流，区别在于，Spark Streaming采用的数据抽象是DStream（本质上就是一系列RDD），而Structured Streaming采用的数据抽象是DataFrame。

- Structured Streaming可以使用Spark SQL的DataFrame/Dataset来处理数据流。虽然Spark SQL也是采用DataFrame作为数据抽象，但是，Spark SQL只能处理静态的数据，而Structured Streaming可以处理结构化的数据流。这样，Structured Streaming就将Spark SQL和Spark Streaming二者的特性结合了起来。

官方User Guide: https://spark.apache.org/docs/2.4.6/structured-streaming-programming-guide.html


```
# 从socket源读取stream
lines = spark\
        .readStream\
        .format('socket')\
        .option('host', 'localhost')\
        .option('port', 9999)\
        .load()
```

```
 # 输出
    query = wordcounts\
            .writeStream\
            .outputMode('complete')\
            .format('console')\
            .trigger(processingTime="8 seconds")\
            .start()
    
```

## File源
File源（或称为“文件源”）以文件流的形式读取某个目录中的文件，支持的文件格式为csv、json、orc、parquet、text等

需要注意的是，文件放置到给定目录的操作应当是原子性的，即不能长时间在给定目录内打开文件写入内容，而是应当采取大部分操作系统都支持的、通过写入到临时文件后移动文件到给定目录的方式来完成

## Kafka源
Kafka源是流处理最理想的输入源，因为它可以保证实时和容错

创建 监控输入终端 监控Kafka收到的文本：

```
bin/kafka-console-consumer.sh \
> --bootstrap-server localhost:9092 --topic wordcount-topic

```

再新开一个终端（记作“监控输出终端”），执行如下命令监控输出的结果文本：
```
bin/kafka-console-consumer.sh --bootstrap-server localhost:9092 --topic wordcount-result-topic

```

# 创建生产者 源源不断地 产生数据

```
import string
import random
import time

from kafka import KafkaProducer


if __name__ == "__main__":
    producer = KafkaProducer(bootstrap_servers=['localhost:9092'])

    while True:
        s2 = (random.choice(string.ascii_lowercase) for _ in range(2))
        word = ''.join(s2)
        value = bytearray(word, 'utf-8')

        producer.send('wordcount-topic', value=value) \
            .get(timeout=10)

        time.sleep(0.1)

```

提交 


```
spark-submit --packages org.apache.spark:spark-sql-kafka-0-10_2.12:2.4.6
```